In [2]:
# you might have to install transfomers and torch
import torch
from transformers import pipeline


In [3]:
my_model = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
my_model('steve jobs is the [MASK] of apple')

[{'score': 0.5164636969566345,
  'token': 3910,
  'token_str': 'founder',
  'sequence': 'steve jobs is the founder of apple'},
 {'score': 0.36049818992614746,
  'token': 5766,
  'token_str': 'ceo',
  'sequence': 'steve jobs is the ceo of apple'},
 {'score': 0.049299854785203934,
  'token': 2343,
  'token_str': 'president',
  'sequence': 'steve jobs is the president of apple'},
 {'score': 0.02111203595995903,
  'token': 8543,
  'token_str': 'creator',
  'sequence': 'steve jobs is the creator of apple'},
 {'score': 0.008550191298127174,
  'token': 2269,
  'token_str': 'father',
  'sequence': 'steve jobs is the father of apple'}]

In [5]:
from slashml import ModelDeployment
import time

In [6]:
# Replace `API_KEY` with your SlasML API token.
client = ModelDeployment(api_key="68b50f46b97f8c0f98fe419df240a22af3da392f")
# client = ModelDeployment(api_key="e7e33ba9a41c1426a425496319a14cab2b3f794f")

In [7]:
response = client.deploy(model_name='my_model_3', model=my_model, requirements=['torch', 'transformers'])

In [9]:
response

{'id': 'c93e32d2-5c52-4caf-8830-e4729eb0b987',
 'created': '2023-09-24T21:06:29.115401Z',
 'status': 'IN_PROGRESS',
 'name': 'my_model_3'}

In [8]:
client.status(model_version_id=response.id)

{'id': 'c93e32d2-5c52-4caf-8830-e4729eb0b987',
 'created': '2023-09-24T21:06:29.115401Z',
 'status': 'IN_PROGRESS',
 'name': 'my_model_3'}

In [10]:
status = client.status(model_version_id=response.id)

while status.status != 'READY':
    print(f'status: {status.status}')
    print('trying again in 5 seconds')
    time.sleep(5)
    status = model.status(model_version_id=response.id)

    if status.status in ['FAILED', "ERROR"]:
        raise Exception(f'Model deployment {status}')

print(f'model succeeded')

model succeeded


In [11]:
input_text = 'Steve jobs is the [MASK] of Apple.'
prediction = client.predict(model_version_id=response.id, model_input=input_text)

{'id': 'c93e32d2-5c52-4caf-8830-e4729eb0b987', 'model_input': 'Steve jobs is the [MASK] of Apple.', 'model_response': {'predictions': [{'score': 0.49993884563446045, 'sequence': 'steve jobs is the founder of apple.', 'token': 3910, 'token_str': 'founder'}, {'score': 0.39265578985214233, 'sequence': 'steve jobs is the ceo of apple.', 'token': 5766, 'token_str': 'ceo'}, {'score': 0.04603331536054611, 'sequence': 'steve jobs is the president of apple.', 'token': 2343, 'token_str': 'president'}, {'score': 0.01692850887775421, 'sequence': 'steve jobs is the creator of apple.', 'token': 8543, 'token_str': 'creator'}, {'score': 0.008718223311007023, 'sequence': 'steve jobs is the chairman of apple.', 'token': 3472, 'token_str': 'chairman'}]}}


In [13]:
prediction['model_response']['predictions']

[{'score': 0.49993884563446045,
  'sequence': 'steve jobs is the founder of apple.',
  'token': 3910,
  'token_str': 'founder'},
 {'score': 0.39265578985214233,
  'sequence': 'steve jobs is the ceo of apple.',
  'token': 5766,
  'token_str': 'ceo'},
 {'score': 0.04603331536054611,
  'sequence': 'steve jobs is the president of apple.',
  'token': 2343,
  'token_str': 'president'},
 {'score': 0.01692850887775421,
  'sequence': 'steve jobs is the creator of apple.',
  'token': 8543,
  'token_str': 'creator'},
 {'score': 0.008718223311007023,
  'sequence': 'steve jobs is the chairman of apple.',
  'token': 3472,
  'token_str': 'chairman'}]